## 교차검증 & 샘플링 & 그리드 서치
- 정확하다! 정확도를 올린다! (X)
- Accuracy Better Model (X) 
- Accuracy Stable Model (O)

In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data') 

### Input & Target로 1차 분리

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy() 

### 세트 분리
- 기존) : train/test 세트로 분리
- 지금) : train/valid/test 세트로 분리

- train/test 8:2로 분리

In [ ]:
from sklearn.model_selection import train_test_split 
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

train_input.shape, test_input.shape, train_target.shape, test_target.shape

((5197, 3), (1300, 3), (5197,), (1300,))

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size = 0.2, random_state=42
)

sub_input.shape, val_input.shape, sub_target.shape, val_target.shape

((4157, 3), (1040, 3), (4157,), (1040,))

- 모형 만들고, 모형 평가
  + 기존) test 데이터를 바로 평가 (unseen data) / 미래에 올 데이터
  + 지금) val 데이터만 평가

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차검증

In [ ]:
from sklearn.model_selection import cross_validate 
scores = cross_validate(dt, train_input, train_target)

print(scores)

{'fit_time': array([0.02020931, 0.02327251, 0.02690148, 0.0244596 , 0.01541948]), 'score_time': array([0.01663065, 0.00184655, 0.00144339, 0.00143909, 0.00143433]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
scores['test_score']

array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])

In [ ]:
import numpy as np

# 검증 데이터 점수
np.mean(scores['test_score'])

0.855300214703487

- StratifiedKFold 활용
  + 통계용어 : 층화추출  (비율에 근거해서 추출하기) 
- 기존 방법 : 임의 추출 방식 (무작위 방식)
- 분류모형
  + 분류기준 / 레드와인 갯수, 화이트와인 갯수
  



In [ ]:
from sklearn.model_selection import StratifiedKFold 
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


- StratifiedKFold() 세부 옵션 지정

In [ ]:
# n_splits : 몇번 교차 검증 할건지 숫자 10
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터의 개념
- 기존) 수동으로 조정, 하나씩 값을 확인하는 형태
  - ridge(), Lasso(), alpha값 조정
  - decision tree, max_depth 값 조정

- 지금) 자동화개발 --> 머신러닝 엔지니어

In [ ]:
from sklearn.model_selection import GridSearchCV

# 모형 만들기
dt = DecisionTreeClassifier(random_state=42)

# 하이퍼파라미터 투입
params = {'max_depth' : [2, 3, 4, 5, 6, 7]}
gs = GridSearchCV(dt, params, n_jobs=-1)

# 모형 학습 --> 7번 수행
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7]})

- 가장 최적화된 max_depth 확인

In [ ]:
print(gs.best_params_)

{'max_depth': 5}


In [ ]:
best_dt = gs.best_estimator_ 
print(best_dt.score(train_input, train_target))

0.8672310948624207


In [ ]:
gs.cv_results_['mean_test_score']

result = pd.DataFrame({
    'max_depth' : [2, 3, 4, 5, 6, 7], 
    'score' : gs.cv_results_['mean_test_score']
})

result

,max_depth,score
0,2,0.813936
1,3,0.841256
2,4,0.853378
3,5,0.857804
4,6,0.855880
5,7,0.855301


In [ ]:
# 하이퍼파라미터 투입
params = {'max_depth' : [2, 3, 4, 5, 6, 7], 
          # 노드 분할 시, 불순도 감소 최저량 지정
          'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.01), 
          'min_samples_split' : range(2, 1000, 10)
          } 

gs = GridSearchCV(dt, params, n_jobs=-1)

# 모형 학습 --> 7번 수행
gs.fit(train_input, train_target)

print(gs.best_params_)

best_dt = gs.best_estimator_ 
print(best_dt.score(train_input, train_target))

{'max_depth': 7, 'min_impurity_decrease': 0.0001, 'min_samples_split': 92}
0.8793534731575909


## 랜덤 서치
- 매개변수 값의 목록을 전달하는 것이 아님
- 매개변수가 샘플링할 수 있는 객체를 전달. 

In [ ]:
from scipy.stats import uniform, randint 

rgen = randint(0, 10)
np.unique(rgen.rvs(100), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([10, 12, 10,  9,  7,  9, 14,  9, 10, 10]))

- 실수 무작위 추출

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.01442643, 0.67060463, 0.92857047, 0.97483125, 0.4672572 ,
       0.8128321 , 0.77896938, 0.59550719, 0.18544863, 0.60533842])

In [29]:
from sklearn.model_selection import RandomizedSearchCV 

# 하이퍼파라미터 투입
params = {'max_depth' : randint(2, 50), 
          # 노드 분할 시, 불순도 감소 최저량 지정
          'min_impurity_decrease' : uniform(0.0001, 0.1), 
          'min_samples_split' : randint(2, 50), 
          }

# 모형 만들기
dt = DecisionTreeClassifier(random_state=42)

# 랜덤서치
rs = RandomizedSearchCV(dt, params, n_iter = 100, random_state=42, n_jobs=-1)

rs.fit(train_input, train_target) 

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7faecf721690>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7faecf721ed0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7faecf6ad290>},
                   random_state=42)

In [30]:
print(rs.best_params_)

best_dt = rs.best_estimator_ 
print(best_dt.score(train_input, train_target))

{'max_depth': 39, 'min_impurity_decrease': 0.00017787658410143285, 'min_samples_split': 22}
0.9053299980758129
